In [1]:
import pandas as pd
import re
from IPython.display import display, HTML
pd.set_option("display.max_rows", None)

In [2]:
df_rdp = pd.read_csv("../datasets/windows/csv/RDP.csv", dtype=str)
df_openssh = pd.read_csv("../datasets/windows/csv/OpenSSH.csv", dtype=str)

In [3]:
# TODO: Implement the code below and run all cells to answer questions 2 - 6

# With reference to our network diagram, we know a list of legitimate client IPs.
# Therefore we could naively filter out these IPs for now.
# HINT: Look at the Network diagram in 1_Context.ipynb

legitimate_client_ip_list = ["192.168.224.127", "192.168.224.128", "192.168.224.129", "192.168.224.1"]

In [4]:
# Run this cell and answer question 2,3,4,5
'''
DO NOT TOUCH THIS SEGMENT! JUST RUN IT!

We are looking for IP addresses that show up in the RDP dataframe that is not part of the network diagram.
'''
useful_rdp_fieldnames = ["System.TimeCreated_attributes.SystemTime",
                    "System.EventID",
                    "UserData.EventXML.Param1",
                    "UserData.EventXML.Param2",
                    "UserData.EventXML.Param3"]
df_rdp_filtered = df_rdp[~df_rdp["UserData.EventXML.Param3"].isin(legitimate_client_ip_list)]
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             df_rdp_filtered[useful_rdp_fieldnames].style.to_html() +
             "</div>"))

,System.TimeCreated_attributes.SystemTime,System.EventID,UserData.EventXML.Param1,UserData.EventXML.Param2,UserData.EventXML.Param3
1,2023-02-10T07:36:27.473976Z,261,nan,nan,nan
2,2023-02-10T07:36:35.219007Z,1149,gerald,nan,192.168.224.130
3,2023-02-10T07:40:11.993075Z,261,nan,nan,nan
4,2023-02-10T07:43:41.371836Z,261,nan,nan,nan
6,2023-02-10T07:52:20.543284Z,261,nan,nan,nan
7,2023-02-10T07:52:25.845624Z,1149,gerald,nan,192.168.224.130
8,2023-02-10T08:01:57.969942Z,261,nan,nan,nan
9,2023-02-10T08:02:03.628777Z,1149,gerald,nan,192.168.224.130


In [5]:
# Run this cell and answer question 6
'''
DO NOT TOUCH THIS SEGMENT! JUST RUN IT!

We are looking for IP addresses that show up in the SSH dataframe that is not part of the network diagram.
'''
def grep_ip_regex(message: str) -> str:
    ip_addr = None
    message_regex = re.compile(r"^.* from (\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3}) .* ?")
    match = message_regex.search(message)
    if match:
        ip_addr = match.group(1)
    return ip_addr

useful_openssh_fieldnames = ["System.TimeCreated_attributes.SystemTime",
                    "System.EventID",
                    "EventData.payload",
                    "Source.IP"]

df_openssh["Source.IP"] = df_openssh["EventData.payload"].apply(lambda x: grep_ip_regex(x))
df_openssh_filtered = df_openssh[~df_openssh["Source.IP"].isin(legitimate_client_ip_list)]
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             df_openssh_filtered[useful_openssh_fieldnames].style.to_html() +
             "</div>"))

,System.TimeCreated_attributes.SystemTime,System.EventID,EventData.payload,Source.IP
0,2023-02-10T07:37:53.410611Z,4,Connection from 192.168.224.130 port 54534 on 192.168.224.137 port 22,192.168.224.130
1,2023-02-10T07:37:53.511651Z,4,Failed publickey for gerald from 192.168.224.130 port 54534 ssh2: ED25519 SHA256:3pzrzdpqM7H0ZBdojFPWpQM5rKrIFt6GdOnrDgqeW3k,192.168.224.130
2,2023-02-10T07:37:53.514570Z,4,Connection closed by authenticating user gerald 192.168.224.130 port 54534 [preauth],None
3,2023-02-10T07:54:41.179273Z,4,Connection from 192.168.224.130 port 54538 on 192.168.224.137 port 22,192.168.224.130
4,2023-02-10T07:54:41.283875Z,4,Accepted key ED25519 SHA256:3pzrzdpqM7H0ZBdojFPWpQM5rKrIFt6GdOnrDgqeW3k found at C:\\Users\\gerald\\.ssh/authorized_keys:2,None
5,2023-02-10T07:54:41.284433Z,4,Postponed publickey for gerald from 192.168.224.130 port 54538 ssh2 [preauth],192.168.224.130
6,2023-02-10T07:54:41.292334Z,4,Accepted key ED25519 SHA256:3pzrzdpqM7H0ZBdojFPWpQM5rKrIFt6GdOnrDgqeW3k found at C:\\Users\\gerald\\.ssh/authorized_keys:2,None
7,2023-02-10T07:54:41.300941Z,4,Accepted publickey for gerald from 192.168.224.130 port 54538 ssh2: ED25519 SHA256:3pzrzdpqM7H0ZBdojFPWpQM5rKrIFt6GdOnrDgqeW3k,192.168.224.130
8,2023-02-10T07:54:41.361092Z,4,User child is on pid 6812,None
9,2023-02-10T07:56:46.430506Z,4,Connection from 192.168.224.130 port 54540 on 192.168.224.137 port 22,192.168.224.130
